In [1]:
!pip install requests
!pip install autopep8
!pip install  urllib
!pip install  streamlit

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


In [2]:
import sys
import os
import requests
import datetime
from  urllib.parse import urlencode
import pandas as pd
import altair as alt
import streamlit as st
import base64

In [3]:
#Auth dashbord https://developer.spotify.com/dashboard/applications
#Env value setting => open jupyter  https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html
client_id = os.environ.get("CLIENTID")
client_secret=os.environ.get("CLIENTSEC")
client_id ="1c87362eaaab48df92d5a6a4ef5f41be"
client_secret="851652f111714e2a959777fccf7ccc58"

In [4]:
client_creds=f"{client_id}:{client_secret}"
type(client_creds)

str

In [5]:
#VALUE: Basic <base64 encoded client_id:client_secret>"
import base64
client_creds=f"{client_id}:{client_secret}"
print(type(client_creds))
     
client_creds_b64=base64.b64encode(client_creds.encode()) 
client_creds_b64

<class 'str'>


b'MWM4NzM2MmVhYWFiNDhkZjkyZDVhNmE0ZWY1ZjQxYmU6ODUxNjUyZjExMTcxNGUyYTk1OTc3N2ZjY2Y3Y2NjNTg='

In [6]:
#Request Access Token
#Flow https://developer.spotify.com/documentation/general/guides/authorization/code-flow/
token_url='https://accounts.spotify.com/api/token'
method="POST"

token_data={
    "grant_type": 'client_credentials'
}

# request must include the following HTTP headers:,Value
#header_parameter

token_headers={
    "Authorization": f"Basic {client_creds_b64.decode()}"#Basic <base64 encoded client_id:client_secret>"
}


In [7]:
r=requests.post(token_url,data=token_data,headers=token_headers)
print(r.json())
valid_request=r.status_code in range(200,299)


{'access_token': 'BQCturDcL9TXQ0PjpN5iMToM_a2L3VFjdX68iIpcGhmWheOFYUEs7aoCKRgRVRVaAtiyQ3akFnPjZnNWfXw', 'token_type': 'Bearer', 'expires_in': 3600}


In [8]:
r=requests.post(token_url,data=token_data,headers=token_headers)
print(r.json())
valid_request=r.status_code in range(200,299)

if valid_request:
    token_response_data=r.json()
    now=datetime.datetime.now()
    access_token=token_response_data["access_token"]
    exprires_in=token_response_data['expires_in']
    expires=now+datetime.timedelta(seconds=exprires_in)
    didi_expires=expires < now



{'access_token': 'BQBy6s13F5pw5npn7rEGJ8zOLQ392klDtMnDAj0Ywsd8uryxCqwfW2mmZp77zCMM5TYXQAWCvuALguxantE', 'token_type': 'Bearer', 'expires_in': 3600}


In [9]:
#class
class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expires=True
    client_id=None
    client_secret=None    
    token_url='https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)#inherit from another class
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        """
        return 64 string
        """
        
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret==None or client_id== None:
            raise Exception("you must set client id and client secret")
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode()) 
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}"#Basic <base64 encoded client_id:client_secret>"
        }
            
    def get_token_data(self):
            return{
    "grant_type": 'client_credentials'
            }
    
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        
        r=requests.post(token_url,data=token_data,headers=token_headers)

        if r.status_code not in range(200,299):
                return False
        token_response_data=r.json()
        now=datetime.datetime.now()
        data=r.json()
        
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
        

In [10]:
spotify=SpotifyAPI(client_id,client_secret)

In [11]:
spotify.perform_auth()

True

In [12]:
access_token=spotify.access_token

In [13]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q": "kill bill", "type": "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

q=kill+bill&type=track
https://api.spotify.com/v1/search?q=kill+bill&type=track
200


In [14]:
r.json()

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=kill+bill&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3KDKJ7Z0kgJrQS1CH43PM4'},
       'href': 'https://api.spotify.com/v1/artists/3KDKJ7Z0kgJrQS1CH43PM4',
       'id': '3KDKJ7Z0kgJrQS1CH43PM4',
       'name': 'Hatesonny',
       'type': 'artist',
       'uri': 'spotify:artist:3KDKJ7Z0kgJrQS1CH43PM4'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      '

In [34]:
#class+serch method
#another search method type https://developer.spotify.com/documentation/web-api/reference/#/operations/search

class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expires=True
    client_id=None
    client_secret=None    
    token_url='https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)#inherit from another class
        self.client_id=client_id
        self.client_secret=client_secret
        
    def get_client_credentials(self):
        """
        return 64 string
        """
        
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret==None or client_id== None:
            raise Exception("you must set client id and client secret")
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode()) 
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64=self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}"#Basic <base64 encoded client_id:client_secret>"
        }
            
    def get_token_data(self):
            return{
    "grant_type": 'client_credentials'
            }
    
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers()
        
        r=requests.post(token_url,data=token_data,headers=token_headers)

        if r.status_code not in range(200,299):
            raise Exception("Not authenticate client")
        token_response_data=r.json()
        now=datetime.datetime.now()
        data=r.json()
        
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
                
        token=self.access_token
        expires=self.access_token_expires
        now=datetime.datetime.now()
        
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token==None:
            self.perform_auth()
            return self.get_access_token
            
        return token
    
    #header
    def get_resource_header(self):
            access_token = self.get_access_token()
            headers = {
                "Authorization": f"Bearer {access_token}"
            }
            return headers

    #depending on input data, get Json
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    #alubum json
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')

    #artist json 
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

    #research Json
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    #query : dic or string  %20 is space
    def search(self,query=None, operator=None,operator_query=None ,search_type='artist'):
        if query == None:
               raise Exception("A query is requred")
               
        if isinstance(query,dict):
               query=" ".join([f"{key},{value}"for key,value in query.items()])#inline 
                
        if operator != None and operator_query!=None:
            operator=operator.upper()
            if operator== "or" or operator.lower=="not":
                operator=operator.upper()
                if isinstance(operator_query,str):
                    query=f"{query}{operator}{operator_query}"
               
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)
    


In [35]:
spotify=SpotifyAPI(client_id,client_secret)

In [36]:
#roubust
spotify.search(query="titanic",operator="NOT",operator_query="Zone",search_type="track")

q=titanic&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=titanic&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4MCBfE4596Uoi2O4DtmEMz'},
       'href': 'https://api.spotify.com/v1/artists/4MCBfE4596Uoi2O4DtmEMz',
       'id': '4MCBfE4596Uoi2O4DtmEMz',
       'name': 'Juice WRLD',
       'type': 'artist',
       'uri': 'spotify:artist:4MCBfE4596Uoi2O4DtmEMz'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ

In [58]:
movie_name="titanic"
data = spotify.search({"album": f"{movie_name}"}, search_type="playlist")
data

q=album%2Ctitanic&type=playlist


{'playlists': {'href': 'https://api.spotify.com/v1/search?query=album%2Ctitanic&type=playlist&offset=0&limit=20',
  'items': [{'collaborative': False,
    'description': '',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/0yWKSWk8QZTXMytKtFqcWg'},
    'href': 'https://api.spotify.com/v1/playlists/0yWKSWk8QZTXMytKtFqcWg',
    'id': '0yWKSWk8QZTXMytKtFqcWg',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b273ccdf9a5a2e2054b0e9d65c04',
      'width': 640}],
    'name': 'Titanic Album',
    'owner': {'display_name': 'Matty',
     'external_urls': {'spotify': 'https://open.spotify.com/user/drmc4m6gejlv86jk3nosyllb9'},
     'href': 'https://api.spotify.com/v1/users/drmc4m6gejlv86jk3nosyllb9',
     'id': 'drmc4m6gejlv86jk3nosyllb9',
     'type': 'user',
     'uri': 'spotify:user:drmc4m6gejlv86jk3nosyllb9'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'MTgsN2U2ZjNhZGE2MjNiNDc1YmVhYzliMDE3NzNlMmE3MDI2N2Q4YTBiNg==',
    

In [56]:
need = []
for i, item in enumerate(data['tracks']['items']):
    track = item['album']
    track_id = item['id']
    song_name = item['name']
    popularity = item['popularity']
    need.append((i, track['artists'][0]['name'], track['name'], track_id, song_name, track['release_date'], popularity))

In [42]:
need

[(0,
  'James Horner',
  "Titanic: Original Motion Picture Soundtrack - Collector's Anniversary Edition",
  '7ICIp5yx2L8x5ymAZugi3D',
  'A Building Panic - Album Suite',
  '1997',
  20),
 (1,
  'James Horner',
  "Titanic: Original Motion Picture Soundtrack - Collector's Anniversary Edition",
  '6qFFrrMou6AEBZse0k9fFK',
  'A Building Panic - Album Suite',
  '1997',
  30),
 (2,
  'James Horner',
  'The Rocketeer (Original Motion Picture Soundtrack)',
  '1fNu1TRJtLJsZ6ksALUh5l',
  'Rocketeer to the Rescue/End Title - 1991 Soundtrack Album/2020 Remaster',
  '2021-05-28',
  17)]

In [43]:
Track_df = pd.DataFrame(need, index=None, columns=('Item', 'Artist', 'Album Name', 'Id', 'Song Name', 'Release Date', 'Popularity'))
Track_df 

,Item,Artist,Album Name,Id,Song Name,Release Date,Popularity
0,0,James Horner,Titanic: Original Motion Picture Soundtrack - ...,7ICIp5yx2L8x5ymAZugi3D,A Building Panic - Album Suite,1997,20
1,1,James Horner,Titanic: Original Motion Picture Soundtrack - ...,6qFFrrMou6AEBZse0k9fFK,A Building Panic - Album Suite,1997,30
2,2,James Horner,The Rocketeer (Original Motion Picture Soundtr...,1fNu1TRJtLJsZ6ksALUh5l,Rocketeer to the Rescue/End Title - 1991 Sound...,2021-05-28,17


In [44]:
access_token = spotify.access_token

headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/audio-features/"

In [45]:
Feat_df = pd.DataFrame()
for id in Track_df['Id'].iteritems():
    track_id = id[1]
    lookup_url = f"{endpoint}{track_id}"
    ra = requests.get(lookup_url, headers=headers)
    audio_feat = ra.json()
    Features_df = pd.DataFrame(audio_feat, index=[0])
    Feat_df = Feat_df.append(Features_df)
Feat_df

/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_3122/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_3122/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)
/var/folders/tg/p4s0x3b55936405b1y1vws6h0000gn/T/ipykernel_3122/2184856597.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Feat_df = Feat_df.append(Features_df)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.0865,0.233,2,-19.175,1,0.0461,0.600,0.775,0.0916,0.0360,63.299,audio_features,7ICIp5yx2L8x5ymAZugi3D,spotify:track:7ICIp5yx2L8x5ymAZugi3D,https://api.spotify.com/v1/tracks/7ICIp5yx2L8x...,https://api.spotify.com/v1/audio-analysis/7ICI...,488933,4
0,0.0859,0.233,10,-19.174,1,0.0462,0.600,0.768,0.0916,0.0361,66.117,audio_features,6qFFrrMou6AEBZse0k9fFK,spotify:track:6qFFrrMou6AEBZse0k9fFK,https://api.spotify.com/v1/tracks/6qFFrrMou6AE...,https://api.spotify.com/v1/audio-analysis/6qFF...,488933,3
0,0.1270,0.208,0,-17.739,1,0.0378,0.675,0.921,0.0851,0.0692,94.606,audio_features,1fNu1TRJtLJsZ6ksALUh5l,spotify:track:1fNu1TRJtLJsZ6ksALUh5l,https://api.spotify.com/v1/tracks/1fNu1TRJtLJs...,https://api.spotify.com/v1/audio-analysis/1fNu...,394274,4


In [62]:
Full_Data = Track_df.merge(Feat_df, left_on="Id", right_on="id")
Full_Data 

,Item,Artist,Album Name,Id,Song Name,Release Date,Popularity,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,James Horner,Titanic: Original Motion Picture Soundtrack - ...,7ICIp5yx2L8x5ymAZugi3D,A Building Panic - Album Suite,1997,20,0.0865,0.233,2,...,0.0916,0.0360,63.299,audio_features,7ICIp5yx2L8x5ymAZugi3D,spotify:track:7ICIp5yx2L8x5ymAZugi3D,https://api.spotify.com/v1/tracks/7ICIp5yx2L8x...,https://api.spotify.com/v1/audio-analysis/7ICI...,488933,4
1,1,James Horner,Titanic: Original Motion Picture Soundtrack - ...,6qFFrrMou6AEBZse0k9fFK,A Building Panic - Album Suite,1997,30,0.0859,0.233,10,...,0.0916,0.0361,66.117,audio_features,6qFFrrMou6AEBZse0k9fFK,spotify:track:6qFFrrMou6AEBZse0k9fFK,https://api.spotify.com/v1/tracks/6qFFrrMou6AE...,https://api.spotify.com/v1/audio-analysis/6qFF...,488933,3
2,2,James Horner,The Rocketeer (Original Motion Picture Soundtr...,1fNu1TRJtLJsZ6ksALUh5l,Rocketeer to the Rescue/End Title - 1991 Sound...,2021-05-28,17,0.1270,0.208,0,...,0.0851,0.0692,94.606,audio_features,1fNu1TRJtLJsZ6ksALUh5l,spotify:track:1fNu1TRJtLJsZ6ksALUh5l,https://api.spotify.com/v1/tracks/1fNu1TRJtLJs...,https://api.spotify.com/v1/audio-analysis/1fNu...,394274,4


In [64]:
Sort_DF = Full_Data.sort_values(by=['Popularity'], ascending=False).head(10)
Sort_DF

,Item,Artist,Album Name,Id,Song Name,Release Date,Popularity,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
1,1,James Horner,Titanic: Original Motion Picture Soundtrack - ...,6qFFrrMou6AEBZse0k9fFK,A Building Panic - Album Suite,1997,30,0.0859,0.233,10,...,0.0916,0.0361,66.117,audio_features,6qFFrrMou6AEBZse0k9fFK,spotify:track:6qFFrrMou6AEBZse0k9fFK,https://api.spotify.com/v1/tracks/6qFFrrMou6AE...,https://api.spotify.com/v1/audio-analysis/6qFF...,488933,3
0,0,James Horner,Titanic: Original Motion Picture Soundtrack - ...,7ICIp5yx2L8x5ymAZugi3D,A Building Panic - Album Suite,1997,20,0.0865,0.233,2,...,0.0916,0.0360,63.299,audio_features,7ICIp5yx2L8x5ymAZugi3D,spotify:track:7ICIp5yx2L8x5ymAZugi3D,https://api.spotify.com/v1/tracks/7ICIp5yx2L8x...,https://api.spotify.com/v1/audio-analysis/7ICI...,488933,4
2,2,James Horner,The Rocketeer (Original Motion Picture Soundtr...,1fNu1TRJtLJsZ6ksALUh5l,Rocketeer to the Rescue/End Title - 1991 Sound...,2021-05-28,17,0.1270,0.208,0,...,0.0851,0.0692,94.606,audio_features,1fNu1TRJtLJsZ6ksALUh5l,spotify:track:1fNu1TRJtLJsZ6ksALUh5l,https://api.spotify.com/v1/tracks/1fNu1TRJtLJs...,https://api.spotify.com/v1/audio-analysis/1fNu...,394274,4


In [48]:
chart_df = Sort_DF[['Artist', 'Album Name', 'Song Name', 'Release Date', 'Popularity',"energy"]]
chart_df 

,Artist,Album Name,Song Name,Release Date,Popularity,energy
1,James Horner,Titanic: Original Motion Picture Soundtrack - ...,A Building Panic - Album Suite,1997,30,0.233
0,James Horner,Titanic: Original Motion Picture Soundtrack - ...,A Building Panic - Album Suite,1997,20,0.233
2,James Horner,The Rocketeer (Original Motion Picture Soundtr...,Rocketeer to the Rescue/End Title - 1991 Sound...,2021-05-28,17,0.208


In [51]:
Name_of_Feat="energy"
st.header(f"Popularity+Energy Chart")
c = alt.Chart(chart_df).mark_circle().encode(
    alt.X('Popularity', scale=alt.Scale(zero=False)), y=f'{Name_of_Feat}', color=alt.Color('Popularity', scale=alt.Scale(type='log',scheme='rainbow')), 
    size=alt.value(300), tooltip=['Popularity', f'{Name_of_Feat}', 'Song Name', 'Album Name']).interactive()

c

alt.Chart(...)

In [52]:
st.header(f"{movie_name}sound track list")
st.table(chart_df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)